In [44]:
# code to access and manipulate data from the https://pota.app/#/ website API

# With help from the team on the POTA Discord API Channel



In [45]:
#---------------imports and setup
import requests
import pandas as pd
import numpy as np
import openpyxl


#setup logging <- setup needs to be specific to jupyter notebook
import logging

#we do this to set up logging so it works in jupyter notebook AND does not spawn duplicate loggers if the cell is run repeatedly
try:
    logger
    
except NameError: #logging not set up yet
    
    logger = logging.getLogger()
    fhandler = logging.FileHandler(filename='python.log', mode='a')
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.setLevel(logging.DEBUG)
    
logging.info(f'starting to log for pota_data')

In [46]:
#----------------Jupyter Notebook Helpers

# display settings 
#see https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython-notebook-in-my-browser

#display full window width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [47]:
#---------------# API Endpoints

# these are taken from the app, or mentioned in the discord. 
# public APIs only, does not include developer APIs requiring authentication

pota_locations_api_url = "https://api.pota.app/locations" 
# index list of State/Region codes although not all locations in that list have parks. (The other url will return an empty list for some of them.)

# https://api.pota.app/location/parks/US-IL 
# where you would replace the US-IL with what every State / Region you want to get parks in. 
# Returns a list of dictionaries / objects. Which each hold the parks reference (park number), Name, Latitude and Longitude, Grid Square, 
# locationDesc (which is useful for trails since it has the list of all regions the park is in), attempts, activations and total QSOs.

# https://api.pota.app/spot/activator
# the list of current POTA activators like what is shown on the POTA spots page.
# count is the number of spots we've had for that station, it's the number that shows up on the table

# https://api.pota.app/park/activations/%7B%7D?count=all
# activations
# %7B%7D is for US locations

# https://api.pota.app/park/<reference> 
# provides grid square, entity, and name, 

# https://api.pota.app/v1/spots
# the new endpoint to get the current spots  

parks_data_csv_url = 'https://pota.app/all_parks_ext.csv'
# For just park information, we do have a CSV available. You can have clients check once a week or on demand to load more info: 



In [48]:
# ----------------------global activation count data

# there is an api which will give y9u the activati9n counts
# but it is available by US State or foreign province/ territorty, there is no global way to access it
# we will consrtuct by getting a list of locations and then querying each entry to get the activqtion totals
# this will be used to make a global list for visualization

logging.info(f'starting to harvest location data')

# get a list of the locations
df_pota_locations = pd.read_json(pota_locations_api_url)
logging.info(f'location data harvested from {pota_locations_api_url}')


# returned data looks like this:
# index locationId entityId  entityName            locationDesc locationName latitude longitude
# 0         2         6       Alaska                  US-AK       Alaska      61.3707 -152.4040
# 1         3        291 United States Of America     US-AL       Alabama     32.8067 -86.7911

# we want the locationDesc which is a globally unique identifier
# plugging this into the API will give us the activations for the entity
pota_loc_desc_names = df_pota_locations["locationDesc"].values.tolist()

# make an empty dataframe to put things into
df_pota_global_activation_count_data = pd.DataFrame()

for pota_location in pota_loc_desc_names :
    
    df_pota_locations_temp = pd.read_json(f'https://api.pota.app/location/parks/{pota_location}')
    df_pota_global_activation_count_data = pd.concat([df_pota_global_activation_count_data, df_pota_locations_temp], sort=False, ignore_index=True)
    logging.info(f'pota_global_activation_count_data data harvested for {pota_location}')

logging.info(f'pota_global_activation_count_data data harvested')

#persist to an excel file
file_save_location = "pota_global_activation_count_data.xlsx"
df_pota_global_activation_count_data.to_excel(file_save_location)
logging.info(f'pota_global_activation_count_data data persisted to {file_save_location}')



    